Rough Heston Call prices
---
***

In [1]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import time
from scipy.optimize import bisect
from scipy.special import gamma
from scipy import integrate

### Black-Scholes and implied volatility

In [2]:
def phi(x): ## Gaussian density
    return np.exp(-x*x/2.)/np.sqrt(2*np.pi)

#### Black Sholes Vega
def BlackScholesVegaCore(DF,F,X,T,v):
    vsqrt=v*np.sqrt(T)
    d1 = (np.log(F/X)+(vsqrt*vsqrt/2.))/vsqrt
    return F*phi(d1)*np.sqrt(T)/DF

#### Black Sholes Function
def BlackScholesCore(CallPutFlag,DF,F,X,T,v):
    ## DF: discount factor
    ## F: Forward
    ## X: strike
    vsqrt=v*np.sqrt(T)
    d1 = (np.log(F/X)+(vsqrt*vsqrt/2.))/vsqrt
    d2 = d1-vsqrt
    if CallPutFlag:
        return DF*(F*norm.cdf(d1)-X*norm.cdf(d2))
    else:
        return DF*(X*norm.cdf(-d2)-F*norm.cdf(-d1))
    
##  Black-Scholes Pricing Function
def BlackScholes(CallPutFlag,S,X,T,r,d,v):
    ## r, d: continuous interest rate and dividend
    return BlackScholesCore(CallPutFlag,np.exp(-r*T),np.exp((r-d)*T)*S,X,T,v)

def impliedvol(price,r,T,s0,K):
    ## Bisection algorithm when the Lee-Li algorithm breaks down
    def smileMin(vol, *args):
        K, s0, T, r, price = args
        return price - BlackScholes(True, s0, K, T, r, 0., vol)
    vMin = 0.000001
    vMax = 10.
    return bisect(smileMin, vMin, vMax, args=(K, s0, T, r, price), rtol=1e-15, full_output=False, disp=True)

## Rough Heston

We consider the following for of rough Heston model:
$d S_t = S_t\sqrt{V_t}d B_t$, with 
$$
V_t = V_0 + \frac{1}{\Gamma(H+\frac{1}{2})}\int_{0}^{t}K(t-u)\Big(\lambda(\theta-V_u)du + \lambda\nu\sqrt{V_u}dW_u\Big),
$$
where the kernel is of the form $K(u) = u^{H-1/2}$, with $H \in (0,1)$.
The two Brownian motions $W$ and $B$ are correlated as $d\langle W,B\rangle_t = \rho d t$, for $\rho \in [-1,1]$.
We also use $\alpha := H+\frac{1}{2}$.

In [3]:
# Heston model class
class Heston_Analytical:

    def __init__(self, dt, heston_params, T):
        # Time discretisation parameters
        self.dt = dt
        self.T = T
        self.n = int(self.T/self.dt)
        self.time_grid = np.linspace(0., T, self.n + 1)

        # Heston model paramters
        self.S0 = heston_params['S0']
        self.Gamma = heston_params['Gamma'] ## Corresponds to the parameter $\lambda$ in the SDE
        self.nu = heston_params['nu']
        self.theta = heston_params['theta']
        self.alpha = heston_params['alpha']
        self.V0 = heston_params['V0']
        self.rho = heston_params['rho']

        # Precomputations to speed up pricing
        self.gam_nu = self.Gamma*self.nu
        self.rho_nu = self.rho*self.nu
        self.frac = self.dt**self.alpha / gamma(self.alpha + 2.)
        self.frac2 = self.dt**self.alpha / gamma(self.alpha + 1.)
        self.frac_bar = 1. / gamma(1 - self.alpha)
        self.fill_a()
        self.fill_b()

    # Fractional Riccati equation
    def F(self, a, x):
        return 0.5*(-a*a - 1j*a) + self.Gamma*(1j*a*self.rho_nu-1.)*x + self.gam_nu*self.gam_nu*x*x/2.

    # Filling the coefficient a and b which don't depend on the characteristic function
    def a(self, j, k):
        if j == 0:
            res = ((k-1.)**(self.alpha+1.) - (k-self.alpha-1.)*k**self.alpha)
        elif j == k:
            res = 1.
        else:
            res = ((k+1.-j)**(self.alpha+1.) + (k-1.-j)**(self.alpha+1.) - 2 * (k-j)**(self.alpha+1.))

        return self.frac*res

    def fill_a(self):
        self.a_ = np.zeros(shape = (self.n+1, self.n+1))
        for k in range(1, self.n + 1):
            for j in range(k+1):
                self.a_[j, k] = self.a(j, k)

    def b(self, j, k):
        return self.frac2 * ((k-j)**self.alpha - (k-j-1.)**self.alpha)

    def fill_b(self):
        self.b_ = np.zeros(shape = (self.n, self.n+1))
        for k in range(1, self.n+1):
            for j in range(k):
                self.b_[j, k] = self.b(j, k)

    # Computation of two sums used in the scheme
    def h_P(self, a, k):
        res = 0
        for j in range(k):
            res += self.b_[j, k] * self.F(a, self.h_hat[j])
        return res

    def sum_a(self, a, k):
        res = 0
        for j in range(k):
            res += self.a_[j, k] * self.F(a, self.h_hat[j])
        return res

    # Solving function h for each time step
    
    def fill_h(self, a):
        #行向量
        self.h_hat = np.zeros((self.n+1), dtype=complex)
        
        for k in range(1, self.n+1):
            h_P = self.h_P(a, k)
            sum_a = self.sum_a(a, k)
            self.h_hat[k] = sum_a + self.a_[k, k]*self.F(a, h_P)

    # Characteristic function computation
    def characteristic_function(self, a):
        # Filling the h function
        self.fill_h(a)

        # Standard integral of the h function
        integral = integrate.trapz(self.h_hat, self.time_grid)

        # Fractional integral of the h function
        func = lambda s: (self.T - s)**(- self.alpha)
        
        #piecewise constant approximation for frac_integral
        y = np.array([func(self.time_grid[i]) * self.h_hat[i] for i in range(self.n)])
        frac_integral = self.frac_bar * np.sum(y) * (self.time_grid[-1] - self.time_grid[-2])

        # Characteristic function
        return np.exp(self.theta * self.Gamma * integral + self.V0 * frac_integral)    
    
    #compute 1st and 2nd order partial derivative of h(a,s)
    def h_pd(self, a, delta):
        #1st order 
        self.fill_h(a + delta/2)
        h_1_a = self.h_hat 
        
        self.fill_h(a - delta/2)
        h_1_b = self.h_hat
        
        p1 = (h_1_a - h_1_b)/delta        
        
        #2nd order 
        self.fill_h(a + delta)
        h_2_a = self.h_hat
        
        self.fill_h(a)
        h_2_b = self.h_hat
        
        self.fill_h(a - delta)
        h_2_c = self.h_hat
        
        p2 = (h_2_a - 2*h_2_b + h_2_c)/(delta**2)
        
        return p1, p2
    
    #compute 1st and 2nd moment of X_i, i=1,...,n, by chara_func 
    def mymoment(self, i):
        p1, p2 = self.h_pd(0, 0.001)
        
        integral_1 = integrate.trapz(p1[:i+1], self.time_grid[:i+1])
        integral_2 = integrate.trapz(p2[:i+1], self.time_grid[:i+1])
        
        func = lambda s: (self.time_grid[i] - s)**(- self.alpha)
        
        y_1 = np.array([func(self.time_grid[j]) * p1[j] for j in range(i)])
        frac_integral_1 = self.frac_bar * np.sum(y_1) * self.dt
        
        y_2 = np.array([func(self.time_grid[j]) * p2[j] for j in range(i)])
        frac_integral_2 = self.frac_bar * np.sum(y_2) * self.dt
        
        G_1 = self.theta * self.Gamma * integral_1 + self.V0 * frac_integral_1
        G_2 = self.theta * self.Gamma * integral_2 + self.V0 * frac_integral_2
        
        
        M_1 = -1j  * G_1
        M_2 = -1 * (G_1**2 + G_2)
        
        return M_1, M_2      
   
    # Pricing with an inverse Fourier transform
    def price_call_rough(self, k):
        K = self.S0*np.exp(k)
        func = lambda u: np.real(np.exp(-1j*u*k)*self.characteristic_function(u - 0.5*1j))/(u**2 + 0.25)
        x = np.linspace(0, 70, 300)
        #从一个可迭代对象创建一个新的数组
        y = np.fromiter((func(xi) for xi in x), dtype=float)
        integ = integrate.trapz(y, x)
        price = self.S0 - 1./np.pi * np.sqrt(self.S0*K) *integ
        iv = impliedvol(price,0.,self.T,self.S0,K)
        return price, iv

    # Analytical formula for the standard Heston characteristic function
    def classical_Heston_characteristic_function(self,u):
        t = self.T
        l = self.nu*self.Gamma
        d = np.sqrt((self.rho*l*u*1j - self.Gamma)**2 + l**2 *(1j*u + u**2))
        g = (self.Gamma - self.rho*l*1j*u - d)/(self.Gamma - self.rho*l*1j*u + d)
        return np.exp(1j*u*(np.log(self.S0)))\
               *np.exp(self.theta*self.Gamma/l**2*((self.Gamma - self.rho*l*1j*u - d)*t - 2*np.log((1. - g*np.exp(-d*t))/(1. - g))))\
               *np.exp(self.V0/l**2*(self.Gamma - self.rho*l*1j*u - d)*(1. - np.exp(-d*t))/(1 - g*np.exp(-d*t)))
    
    # Pricing with an inverse Fourier transform
    def price_call_classical(self, k):
        K = self.S0 * np.exp(k)
        func = lambda u: np.real (np.exp (-1j * u * k) * self.classical_Heston_characteristic_function (u - 0.5 * 1j)) / (u ** 2 + 0.25)
        x = integrate.quad(func, 0, np.inf)
        self.classicalPrice = self.S0 - 1 / np.pi * np.sqrt (self.S0 * K) * x[0]
        return self.classicalPrice

In [4]:
# Heston parameters
Gamma = .1
nu = .331
alpha = 0.62
rho = -0.01
V0 = 0.0392
theta = 0.3156
S0 = 1.
dt = 0.01
T = 0.5
heston_params = {'Gamma': Gamma, 'nu': nu, 'alpha': alpha, 'rho': rho, 'V0': V0, 'theta': theta, 'S0': S0}

In [5]:
he = Heston_Analytical(dt, heston_params, T)

In [6]:
#compute 1st and 2nd moment of X_1, X_2,...,X_n
moment = np.real(np.array([[he.mymoment(i)[0] for i in range(1,he.n+1)], [he.mymoment(j)[1] for j in range(1, he.n+1)]]))
np.set_printoptions(suppress=True)
print(moment)

[[-0.00000505 -0.00011102 -0.00023896 -0.00038062 -0.00053231 -0.00069197
  -0.00085827 -0.0010303  -0.00120741 -0.00138908 -0.00157492 -0.0017646
  -0.00195786 -0.00215447 -0.00235425 -0.00255702 -0.00276265 -0.00297101
  -0.00318198 -0.00339547 -0.00361138 -0.00382963 -0.00405015 -0.00427286
  -0.00449771 -0.00472463 -0.00495358 -0.0051845  -0.00541734 -0.00565206
  -0.00588863 -0.00612699 -0.00636712 -0.00660898 -0.00685254 -0.00709776
  -0.00734463 -0.0075931  -0.00784316 -0.00809478 -0.00834794 -0.0086026
  -0.00885876 -0.00911638 -0.00937546 -0.00963596 -0.00989787 -0.01016117
  -0.01042585 -0.01069189]
 [ 0.0000101   0.00022206  0.00047798  0.0007614   0.00106494  0.00138446
   0.00171734  0.00206176  0.0024164   0.00278024  0.0031525   0.00353253
   0.00391981  0.00431389  0.00471439  0.00512099  0.00553339  0.00595135
   0.00637466  0.00680309  0.00723649  0.00767468  0.00811753  0.00856488
   0.00901663  0.00947265  0.00993284  0.0103971   0.01086534  0.01133748
   0.01181343

In [8]:
moment.tofile("ref_moment.bin")

In [ ]:
logMoneynesses = np.linspace(-0.25, 0.25, 10)
strikes = [S0*np.exp(k) for k in logMoneynesses]

he = Heston_Analytical(dt, heston_params, T)
ccs, ivs = [], []
for k in logMoneynesses:
    c, v = he.price_call_rough(k)
    ccs.append(c)
    ivs.append(v)

plt.plot(logMoneynesses, ccs, 'b', label='alpha= %.1f' %he.alpha)
plt.legend(loc="best")
plt.xlabel("log-strike")
plt.title("Call prices")
plt.show()

plt.plot(logMoneynesses, ivs, 'b', label='alpha= %.1f' %he.alpha)
plt.legend(loc="best")
plt.xlabel("log-strike")
plt.title("Implied volatility")
plt.show()